In [6]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler

In [8]:
test_df = pd.read_csv('./dataset/Test_dataset.csv')
test_df = test_df[['uuid','X_LOC', 'Y_LOC', 'DEPT', 'NPHI','DTC', 'RHOB', 'GR','CALI']]
test_df

,uuid,X_LOC,Y_LOC,DEPT,NPHI,DTC,RHOB,GR,CALI
0,14812,459853.34375,6560993.0,1348.310400,0.398266,133.537033,2.131319,61.177399,12.301304
1,72908,459853.34375,6560993.0,1348.462400,0.389460,133.525543,2.129777,63.512333,12.299226
2,103104,459853.34375,6560993.0,1348.614400,0.394868,130.739624,2.138082,63.515835,12.297644
3,23480,459853.34375,6560993.0,1348.766400,0.389355,128.074249,2.153999,63.153057,12.276056
4,104267,459853.34375,6560993.0,1348.918400,0.365808,121.454926,2.140920,60.224148,12.299644
...,...,...,...,...,...,...,...,...,...
121792,10928,NaN,NaN,5006.809976,NaN,NaN,2.515146,65.925987,8.647468
121793,26727,NaN,NaN,5006.961976,NaN,NaN,2.543073,66.297127,8.636636
121794,83995,NaN,NaN,5007.113976,NaN,NaN,2.576337,64.853714,8.592650
121795,67375,NaN,NaN,5007.265976,NaN,NaN,2.606787,62.779541,8.546233


In [13]:
# Definisi hubungan fitur untuk imputasi
imputation_dict = {
    'NPHI': ['RHOB', 'DTC', 'DEPT'],
    'DTC': ['RHOB', 'NPHI', 'DEPT'],
    'GR': ['DEPT'],
    'CALI': ['DEPT'],
    'RHOB': ['NPHI', 'DTC', 'DEPT'],
}

def impute_feature(df, target_feature, predictor_features):
    # Periksa apakah kolom target 100% NaN
    if df[target_feature].isna().sum() == len(df):
        print(f"Skipping {target_feature}: 100% NaN.")
        return df  # Kembalikan DataFrame tanpa perubahan
    
    # Periksa apakah semua kolom prediktor 100% NaN
    valid_predictors = [col for col in predictor_features if df[col].isna().sum() < len(df)]
    
    if len(valid_predictors) == 0:
        print(f"Skipping {target_feature}: All predictor columns are 100% NaN.")
        return df  # Kembalikan DataFrame tanpa perubahan

    # Cek apakah ada NaN yang perlu diimputasi di target
    if df[target_feature].isna().sum() > 0:
        print(f"Imputing {target_feature} using {valid_predictors}...")

        # Ambil subset data dengan fitur prediktor valid dan target
        imputation_data = df[valid_predictors + [target_feature]].copy()

        # Standarisasi data
        scaler = StandardScaler()
        imputation_data_scaled = scaler.fit_transform(imputation_data)

        # Terapkan KNN Imputer
        imputer = KNNImputer(n_neighbors=5, weights='distance')
        imputed_data_scaled = imputer.fit_transform(imputation_data_scaled)

        # Kembalikan ke skala asli
        imputed_data = scaler.inverse_transform(imputed_data_scaled)
        imputed_df = pd.DataFrame(imputed_data, columns=valid_predictors + [target_feature], index=df.index)

        # Update DataFrame asli
        df[target_feature] = imputed_df[target_feature]
    else:
        print(f"No NaN values in {target_feature}. Skipping imputation.")
    
    return df

In [14]:
# Create a copy of test_df to work with
df = test_df.copy()

# Lakukan imputasi untuk setiap fitur target
for target_feature, predictor_features in imputation_dict.items():
    # Pastikan semua fitur prediktor ada di DataFrame
    if all(feature in df.columns for feature in predictor_features):
        df = impute_feature(df, target_feature, predictor_features)
    else:
        print(f"Skipping imputation for {target_feature}")

Imputing NPHI using ['RHOB', 'DTC', 'DEPT']...
Imputing DTC using ['RHOB', 'NPHI', 'DEPT']...
Imputing GR using ['DEPT']...
Imputing CALI using ['DEPT']...
Imputing RHOB using ['NPHI', 'DTC', 'DEPT']...


In [16]:
df.fillna(0,inplace=True)

In [18]:
from pycaret.classification import load_model
from pycaret.classification import *

# Muat kembali model
loaded_model = load_model('best_model')

# Gunakan model yang telah dimuat kembali untuk prediksi
test_predictions = predict_model(loaded_model, data=df)

Transformation Pipeline and Model Successfully Loaded


In [22]:
test_predictions

,uuid,X_LOC,Y_LOC,DEPT,NPHI,DTC,RHOB,GR,CALI,prediction_label,prediction_score
0,14812,459853.34375,6560993.0,1348.310425,0.398266,133.537033,2.131319,61.177399,12.301304,30000,0.34
1,72908,459853.34375,6560993.0,1348.462402,0.389460,133.525543,2.129777,63.512333,12.299226,30000,0.37
2,103104,459853.34375,6560993.0,1348.614380,0.394868,130.739624,2.138082,63.515835,12.297644,30000,0.39
3,23480,459853.34375,6560993.0,1348.766357,0.389355,128.074249,2.153999,63.153057,12.276056,30000,0.36
4,104267,459853.34375,6560993.0,1348.918457,0.365808,121.454926,2.140920,60.224148,12.299644,30000,0.31
...,...,...,...,...,...,...,...,...,...,...,...
121792,10928,0.00000,0.0,5006.810059,0.231205,80.453865,2.515146,65.925987,8.647468,65030,0.50
121793,26727,0.00000,0.0,5006.961914,0.232819,80.927139,2.543073,66.297127,8.636636,65030,0.48
121794,83995,0.00000,0.0,5007.113770,0.260863,80.394623,2.576337,64.853714,8.592650,65030,0.50
121795,67375,0.00000,0.0,5007.266113,0.212640,83.122765,2.606787,62.779541,8.546233,65030,0.49


In [24]:
import pandas as pd

# Data mapping berdasarkan label_code dan label_name
lithology_mapping = {
    30000: "Sandstone",
    65030: "Sandstone/Shale",
    65000: "Shale",
    80000: "Marl",
    74000: "Dolomite",
    70000: "Limestone",
    70032: "Chalk",
    88000: "Halite",
    86000: "Anhydrite",
    99000: "Tuff",
    90000: "Coal",
    93000: "Basement"
}


# Buat kolom baru berdasarkan mapping
test_predictions['label_name'] = test_predictions['prediction_label'].map(lithology_mapping)

test_predictions

,uuid,X_LOC,Y_LOC,DEPT,NPHI,DTC,RHOB,GR,CALI,prediction_label,prediction_score,lithology_name,label_name
0,14812,459853.34375,6560993.0,1348.310425,0.398266,133.537033,2.131319,61.177399,12.301304,30000,0.34,Sandstone,Sandstone
1,72908,459853.34375,6560993.0,1348.462402,0.389460,133.525543,2.129777,63.512333,12.299226,30000,0.37,Sandstone,Sandstone
2,103104,459853.34375,6560993.0,1348.614380,0.394868,130.739624,2.138082,63.515835,12.297644,30000,0.39,Sandstone,Sandstone
3,23480,459853.34375,6560993.0,1348.766357,0.389355,128.074249,2.153999,63.153057,12.276056,30000,0.36,Sandstone,Sandstone
4,104267,459853.34375,6560993.0,1348.918457,0.365808,121.454926,2.140920,60.224148,12.299644,30000,0.31,Sandstone,Sandstone
...,...,...,...,...,...,...,...,...,...,...,...,...,...
121792,10928,0.00000,0.0,5006.810059,0.231205,80.453865,2.515146,65.925987,8.647468,65030,0.50,Sandstone/Shale,Sandstone/Shale
121793,26727,0.00000,0.0,5006.961914,0.232819,80.927139,2.543073,66.297127,8.636636,65030,0.48,Sandstone/Shale,Sandstone/Shale
121794,83995,0.00000,0.0,5007.113770,0.260863,80.394623,2.576337,64.853714,8.592650,65030,0.50,Sandstone/Shale,Sandstone/Shale
121795,67375,0.00000,0.0,5007.266113,0.212640,83.122765,2.606787,62.779541,8.546233,65030,0.49,Sandstone/Shale,Sandstone/Shale


In [25]:
submission = test_predictions[['uuid','label_name']]

In [26]:
submission.to_csv("submission.csv",index=False)